Importo las librerías

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os

MyModel se utiliza para definir un modelo de lenguaje recurrente (RNN) básico en TensorFlow usando Keras. Este tipo de modelo se suele utilizar para tareas relacionadas con el procesamiento de lenguaje natural, en este caso, de generación de texto.

In [2]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

OneStep se utiliza para definir un modelo de generación de texto de un solo paso basado en un modelo más grande. Genera un único carácter en función de una entrada dada.

In [8]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

Se cargan los modelos desde google drive o la carpeta

In [26]:
one_step_abascal = tf.saved_model.load('one_step_abascal')
one_step_casado = tf.saved_model.load('one_step_casado')
one_step_sanchez = tf.saved_model.load('one_step_sanchez')

La función encapsula la lógica de generación de texto utilizando un modelo dado. Se puede utilizar esta función para generar respuestas de texto dada una semilla y la longitud deseada.

In [27]:
def generar_respuesta(modelo, semilla, longitud=300):
    states = None
    next_char = tf.constant([semilla])
    result = [next_char]

    for _ in range(longitud):
        next_char, states = modelo.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    respuesta_completa = result[0].numpy().decode('utf-8')

    # Obtener el primer índice después de la semilla
    primer_indice_despues_de_semilla = len(semilla)

    # Obtener todos los caracteres a partir del primer índice después de la semilla
    caracteres_desde_semilla = respuesta_completa[primer_indice_despues_de_semilla:]

    return respuesta_completa, caracteres_desde_semilla




In [29]:
# Ejemplo de uso
respuesta, caracteres_mostrados = generar_respuesta(one_step_casado, 'Señor')
print(f"Respuesta completa: {respuesta}")
print(f"\n\nCaracteres desde el primer carácter después de la semilla: {caracteres_mostrados}")

Respuesta completa: Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima proponemos pagar de inmediato los ERTE pendientes y ampliaron tearquier y a los radicales que incendian Barcelona una vez más. Menudo reencuentro. ¿Cuándo piensa reconocer algún error y pedir disculpas por qué 


Caracteres desde el primer carácter después de la semilla:  Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima proponemos pagar de inmediato los ERTE pendientes y ampliaron tearquier y a los radicales que incendian Barcelona una vez más. Menudo reencuentro. ¿Cuándo piensa reconocer algún error y pedir disculpas por qué 



Aqui se inicia un diálogo utilizando una semilla y los 3 modelos anteriores(one_step_abascal, one_step_casado, one_step_sanchez). Luego, realiza un bucle para continuar el diálogo durante un número especificado de iteraciones.


In [30]:
# Iniciar diálogo
semilla_inicial = 'buenos días'
num_iteraciones = 2  # Puedes ajustar este valor según tus necesidades
print(f"Semilla: {semilla_inicial}")
# Respuestas iniciales
respuesta_abascal, caracteres_mostrados = generar_respuesta(one_step_abascal, 'buenos días')
print(f"------\n Respuesta de Abascal: {respuesta_abascal}")

# Bucle para continuar el diálogo
for i in range(num_iteraciones):
    start_time = time.time()

    print(f"\n-----------Iteración:{num_iteraciones}")

    respuesta_casado, caracteres_mostrados = generar_respuesta(one_step_casado, respuesta_abascal)
    print(f"\n------Respuesta de Casado: {caracteres_mostrados}")

    respuesta_sanchez, caracteres_mostrados = generar_respuesta(one_step_sanchez, respuesta_casado)
    print(f"\n------Respuesta de Sanchez: {caracteres_mostrados}")

    respuesta_abascal, caracteres_mostrados = generar_respuesta(one_step_abascal, respuesta_sanchez)
    print(f"\n------Respuesta de Abascal: {caracteres_mostrados}")

    end_time = time.time()
    
    print('\nRun time:', end_time - start_time)






Semilla: buenos días
------
 Respuesta de Abascal: buenos días. Señor Sánchez, ¿se reunió el ministro de Transporte de contral ros por bor adomas, con la lasie se enfravie tembién da la Corandi cod el nonos de der cobraro que no es lo que hoy a social hon perdido el contacto con la calle, que no están en los mercados, en los bares. Porque si estuvieran, sabría

-----------Iteración:2

------Respuesta de Casado:  Tonal. Dice que va a poner peajes en las carreteras. Por cierto, ¿puede confirmar si eso es la población de anteayer, podemos hacerlo.
Acabo, señora presidenta. … la Usted ha conseguido una investidura, podrá conseguir los presupuestos, podrá conseguir varias leyes ―sobre más escándalos —‌y acabo y

------Respuesta de Sanchez:  tendido. Y tirme de la mayoría parlamentaria en Madrid, la comunidad le tiendo la manoría de su grupo parlamentario. De hecho, creo que todos podemos convenir en que hemos logrado en contra de los jubito en el Primer lugar, demper delante médita que el G